In [70]:
import tkinter as tk
from tkinter import simpledialog, filedialog
from PIL import Image, ImageDraw, ImageFont, ImageTk, ImageGrab
import img2pdf
import io
import os
from reportlab.pdfgen import canvas as pdf_canvas
from reportlab.lib.pagesizes import letter


In [ ]:
class DraggableEntry(tk.Entry):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self.bind("<Button-1>", self.on_click)
        self.bind("<B1-Motion>", self.on_drag)

    def on_click(self, event):
        self.x = event.x
        self.y = event.y

    def on_drag(self, event):
        x = self.winfo_x() + event.x - self.x
        y = self.winfo_y() + event.y - self.y
        self.place(x=x, y=y)

class DraggableArrow:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.arrow = canvas.create_line(x, y, x + 50, y + 50, arrow=tk.LAST, fill="black")
        self.start_circle = canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="blue")
        self.end_circle = canvas.create_oval(x + 47, y + 47, x + 53, y + 53, fill="red")

        canvas.tag_bind(self.start_circle, "<Button-1>", self.start_click)
        canvas.tag_bind(self.start_circle, "<B1-Motion>", self.start_drag)
        canvas.tag_bind(self.end_circle, "<Button-1>", self.end_click)
        canvas.tag_bind(self.end_circle, "<B1-Motion>", self.end_drag)

    def start_click(self, event):
        self.x = event.x
        self.y = event.y

    def start_drag(self, event):
        dx = event.x - self.x
        dy = event.y - self.y
        self.canvas.move(self.start_circle, dx, dy)
        self.canvas.coords(self.arrow, event.x, event.y, *self.canvas.coords(self.arrow)[2:])
        self.x = event.x
        self.y = event.y

    def end_click(self, event):
        self.x = event.x
        self.y = event.y

    def end_drag(self, event):
        dx = event.x - self.x
        dy = event.y - self.y
        self.canvas.move(self.end_circle, dx, dy)
        self.canvas.coords(self.arrow, *self.canvas.coords(self.arrow)[:2], event.x, event.y)
        self.x = event.x
        self.y = event.y

def export_canvas_to_pdf(tk_canvas, pdf_file):
    # Update the Tkinter GUI to ensure all elements are drawn
    tk_canvas.update_idletasks()

    # Get canvas coordinates
    x1 = tk_canvas.winfo_rootx()
    y1 = tk_canvas.winfo_rooty()
    x2 = x1 + tk_canvas.winfo_width()
    y2 = y1 + tk_canvas.winfo_height()
    
    # Capture the canvas as an image using ImageGrab
    canvas_image = ImageGrab.grab().crop((x1, y1, x2, y2))

    # Save the image as a temporary file
    temp_image_file = "temp_canvas_image.png"
    canvas_image.save(temp_image_file, format="PNG")

    # Create a new PDF canvas
    pdf = pdf_canvas.Canvas(pdf_file, pagesize=letter)

    # Draw the image on the PDF canvas
    pdf.drawImage(temp_image_file, 50, 200, 500, 250)

    # Save the PDF
    pdf.save()

    # Remove the temporary image file
    os.remove(temp_image_file)

    
def rounded_rect(canvas, x1, y1, x2, y2, radius=25, **kwargs):
    points = [
        x1 + radius, y1,
        x2 - radius, y1,
        x2, y1, x2, y1 + radius,
        x2, y2 - radius,
        x2, y2, x2 - radius, y2,
        x1 + radius, y2,
        x1, y2, x1, y2 - radius,
        x1, y1 + radius,
        x1, y1, x1 + radius, y1
    ]
    return canvas.create_polygon(points, **kwargs)

def set_canvas_title(canvas, title_var):
    title = simpledialog.askstring("Canvas Title", "Enter the canvas title:")
    if title:
        title_var.set(title)
        canvas.itemconfig(canvas_title, text=title)
        
def export_button_click():
    pdf_file = filedialog.asksaveasfilename(defaultextension=".pdf",
                                            filetypes=[("PDF Files", "*.pdf"), ("All Files", "*.*")])
    if pdf_file:
        export_canvas_to_pdf(canvas, pdf_file)

        
def main():
    global root
    root = tk.Tk()
    root.title("Business Model Canvas")

    canvas = tk.Canvas(root, width=1200, height=600, bg="white")
    canvas.pack()

    # Add title to the canvas
    title_var = tk.StringVar()
    title_var.set("Business Model Canvas")
    canvas_title = canvas.create_text(600, 15, text=title_var.get(), font=("Arial", 14, "bold"))

    
    sections = [
        ("Key Partners", 50, 50, 250, 300),
        ("Key Activities", 250, 50, 450, 175),
        ("Key Resources", 250, 175, 450, 300),
        ("Value Propositions", 450, 50, 650, 300),
        ("Customer Relationships", 650, 50, 850, 175),
        ("Channels", 650, 175, 850, 300),
        ("Customer Segments", 850, 50, 1050, 300),
        ("Cost Structure", 50, 300, 525, 550),
        ("Revenue Streams", 525, 300, 1050, 550),
    ]
    
    
    
    for title, x1, y1, x2, y2 in sections:
        rounded_rect(canvas, x1, y1, x2, y2, radius=25, fill="#F0F0F0", outline="#C0C0C0", width=2)
        canvas.create_text((x1 + x2) // 2, y1 + 25, text=title, font=("Arial", 14, "bold"))

        for i in range(3):
            entry = DraggableEntry(root, width=15)
            entry.insert(0, f"{title} {i + 1}")
            entry.place(x=x1 + 10, y=y1 + 50 + i * 25)

    add_arrow_button = tk.Button(root, text="Add Arrow", command=lambda: DraggableArrow(canvas, 100, 400))
    add_arrow_button.place(x=1000, y=560)
    
    # Add an export button
    export_button = tk.Button(root, text="Export to PDF", command=export_button_click)
    export_button.place(x=1100, y=560)
    
    root.mainloop()

if __name__ == "__main__":
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/szalam/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/24/0hnc07_91nndxxzhr7g6vpvm0000gq/T/ipykernel_13766/3367333449.py", line 106, in export_button_click
    export_canvas_to_pdf(canvas, pdf_file)
  File "/var/folders/24/0hnc07_91nndxxzhr7g6vpvm0000gq/T/ipykernel_13766/3367333449.py", line 57, in export_canvas_to_pdf
    x1 = tk_canvas.winfo_rootx()
  File "/Users/szalam/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1171, in winfo_rootx
    self.tk.call('winfo', 'rootx', self._w))
_tkinter.TclError: can't invoke "winfo" command: application has been destroyed
